# Big Data Management Project 2:
## DESB GRAND CHALLENGE 2015

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, regexp_extract, col, count, udf, window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType, FloatType

from pyspark.sql import functions as F
from pyspark.sql.window import Window

import math
import time



In [2]:
spark = SparkSession.builder \
    .appName('BDM_Project2') \
    .getOrCreate()

### Query 0
Data Cleansing and Setup

In [3]:
start_time = time.time()  

# Defining the schema for faster reading of data
schema = StructType([
    StructField("medallion", StringType(), True),
    StructField("hack_license", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("trip_time_in_secs", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("surcharge", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True)
])

# Creating a single dataframe of all the trip_data files
taxi_df_og = (
    spark.readStream
    .option("maxFilesPerTrigger", 1)
    .option("header", False)
    .schema(schema)
    .csv("input")
)

# Removing the trips with 0 passengers
# Transforming the data 
taxi_df = taxi_df_og.filter(
    (regexp_extract(col("medallion"), r"^[a-fA-F0-9]{32}$", 0) != "") &
    (regexp_extract(col("hack_license"), r"^[a-fA-F0-9]{32}$", 0) != "") &
    (col("pickup_datetime").isNotNull()) &
    (col("dropoff_datetime").isNotNull()) &               
    (col("trip_distance") > 0) &                    
    (col("fare_amount") > 0) &
    (col("tip_amount") >= 0)
)

# Convert timestamps to Unix format 
taxi_df = taxi_df.withColumn("pickup_ts", unix_timestamp("pickup_datetime")) \
    .withColumn("dropoff_ts", unix_timestamp("dropoff_datetime")) \
    .withColumn("duration", col("dropoff_ts") - col("pickup_ts")) \
    .select("*") \
    .dropna()  # Drop remaining null values

# Start the streaming query with trigger(once=True) to process data once and stop
query = (
    taxi_df.writeStream
    .outputMode("append")
    .format("parquet")
    .option("path", "output/preprocessed_data")
    .option("checkpointLocation", "output/checkpoint")
    .trigger(once=True)  
    .start()
)

query.awaitTermination()

print("Execution time", time.time() - start_time)

Execution time 3.028912305831909


In [4]:
output_df = spark.read.parquet("output/preprocessed_data")
output_df.show(5)

+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+----------+----------+--------+
|           medallion|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount| pickup_ts|dropoff_ts|duration|
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+----------+----------+--------+
|64187531006B3D8B3...|871C25EC0B8BF4BC8...|2013-08-01 17:47:36|2013-08-01 19:05:40|             4683|         18.0|      -74.005295|      40.750935|      

In [5]:
# Impact of transformations
#original_count = output_df.count()
#filtered_count = taxi_df.count()
#filtered_out_count = original_count - filtered_count

#print(f"Original count: {original_count}") 
#print(f"Filtered count: {filtered_count}")
#print(f"Rows filtered out: {filtered_out_count}")

taxi_df = (spark.readStream
  .option("maxFilesPerTrigger", 1)
  .schema(schema)
  .parquet("output/preprocessed_data")
)

### Grid Cells for Query 1

In [8]:
start_lat = 41.474937
start_long = -74.913585
cell_size = 0.044 # 500m to degrees for latitude (and longitude)

def grid_cells_q1(point_long, point_lat):

    # calculating the point values for latitude and longitude
    long = math.floor((point_long - start_long) / cell_size) + 1
    lat = math.floor((start_lat - point_lat) / cell_size) + 1
    
    # Ensure the cell is within valid grid bounds (300x300)
    if not (1 <= long <= 300 and 1 <= lat <= 300):
        return None 
    
    return float(f"{long}.{lat}") # Convert to X.X format

get_grid = udf(grid_cells_q1, FloatType())

In [9]:
from pyspark.sql.functions import col, window, count, date_format
from pyspark.sql.streaming import DataStreamWriter
import pandas as pd
from IPython.display import display, Markdown

# Reading in the stream
taxi_df = (spark.readStream
  .option("maxFilesPerTrigger", 1)
  .schema(schema)
  .parquet("output/preprocessed_data")
)

# Query 1: create a query to find the top most frequent routes during the last 30 minutes (Show only the 10 most frequent routes)
# The output query results must be: start_cell, end_cell, Number of Rides
# Aid from ChatGPT was used for the following code

# Adding start_cell and end_cell columns
taxi_df_q1 = taxi_df.withColumn("start_cell", get_grid(taxi_df.pickup_longitude, taxi_df.pickup_latitude))\
    .withColumn("end_cell", get_grid(taxi_df.dropoff_longitude, taxi_df.dropoff_latitude))\
    .filter(
        col("start_cell").isNotNull() & col("end_cell").isNotNull()  # Filter out trips outside of the grid
    )

# Defining a 30-minute tumbling window and aggregating by start_cell and end_cell
top_routes_df = (
    taxi_df_q1
    .withWatermark("dropoff_datetime", "30 minutes") 
    .groupBy(window(col("dropoff_datetime"), "30 minutes"), col("start_cell"), col("end_cell"))
    .agg(count("*").alias("Number of Rides"))
)

# Function for processing the data in batches
def process_batch(df, epoch_id):
    
    df = df.orderBy(col("window.start").desc(), col("Number of Rides").desc())

    # Collect unique time windows
    windows = df.select("window").distinct().orderBy("window.start").collect()
    
    # Finding the top routes for each "last 30 minutes"
    for window_row in windows:
        
        time_window = window_row["window"]
        start_time = time_window.start.strftime("%Y-%m-%d %H:%M:%S")
        end_time = time_window.end.strftime("%Y-%m-%d %H:%M:%S")

        display(Markdown(f"Time Window: {start_time} to {end_time}"))
        # Filter the top 10 rides for the given window
        top_routes = (
            df.filter(col("window.start") == time_window.start)
            .select("start_cell", "end_cell", "Number of Rides")
            .orderBy(col("Number of Rides").desc())
            .limit(10)
            .toPandas()  # Using to Pandas DataFrame to display the results in the notebook
        )

        display(top_routes)

# Streaming query
query: DataStreamWriter = (
    top_routes_df
    .writeStream
    .outputMode("update")
    .foreachBatch(process_batch)
    .start()
)

query.awaitTermination()

Time Window: 2013-01-04 11:30:00 to 2013-01-04 12:00:00

,start_cell,end_cell,Number of Rides
0,22.17,22.17,514
1,22.17,21.17,191
2,22.16,22.17,190
3,22.16,22.16,183
4,21.17,22.17,163
5,21.17,21.17,152
6,22.17,22.16,150
7,22.17,21.18,69
8,21.18,21.18,61
9,21.18,21.17,59


Time Window: 2013-01-04 12:00:00 to 2013-01-04 12:30:00

,start_cell,end_cell,Number of Rides
0,22.17,22.17,1112
1,22.16,22.17,423
2,22.17,21.17,410
3,22.16,22.16,374
4,21.17,22.17,348
5,22.17,22.16,346
6,21.17,21.17,345
7,22.17,21.18,153
8,21.18,21.17,151
9,21.18,22.17,140


Time Window: 2013-01-04 12:30:00 to 2013-01-04 13:00:00

...

KeyboardInterrupt: 

,start_cell,end_cell,Number of Rides
0,22.17,22.17,804
1,22.16,22.16,445
2,22.17,22.16,429
3,22.17,21.17,376
4,21.17,21.17,325
5,21.17,22.17,304
6,22.16,22.17,273
7,22.17,21.18,125
8,21.17,21.18,109
9,21.18,21.17,103


Time Window: 2013-01-04 16:30:00 to 2013-01-04 17:00:00

,start_cell,end_cell,Number of Rides
0,22.17,22.17,707
1,22.17,22.16,395
2,22.16,22.16,394
3,22.17,21.17,323
4,21.17,21.17,310
5,21.17,22.17,306
6,22.16,22.17,280
7,21.18,21.17,123
8,21.18,22.17,109
9,21.17,21.18,105


Time Window: 2013-01-04 17:00:00 to 2013-01-04 17:30:00

In [10]:
query.stop()

### Query 1
Frequent Routes

### Grid Cells for Query 2

In [13]:
# Imports 
from pyspark.sql.functions import col, expr, percentile_approx, lit, udf, max, sum, window, lead
from pyspark.sql.types import StringType
from pyspark.sql.streaming import DataStreamWriter

start_lat = 41.474937
start_long = -74.913585
cell_size = 0.022 # 250m to degrees for latitude (and longitude)

def grid_cells_q2(point_long, point_lat):
    
    long = (point_long - start_long) // cell_size + 1
    lat = (start_lat - point_lat) // cell_size + 1
    
    # Ensure the cell is within valid grid bounds (600x600)
    if not (1 <= long <= 600 and 1 <= lat <= 600):
        return None 
    
    return f"{long}.{lat}" # Convert to X.X format

get_grid2 = udf(grid_cells_q2, FloatType())

In [14]:
# Reading in the stream
taxi_df = (spark.readStream
  .option("maxFilesPerTrigger", 1)
  .schema(schema)
  .parquet("output/preprocessed_data")
)

# Adding start_cell and end_cell columns
taxi_df_q2 = taxi_df.withColumn("start_cell", get_grid2(taxi_df.pickup_longitude, taxi_df.pickup_latitude))\
    .withColumn("end_cell", get_grid2(taxi_df.dropoff_longitude, taxi_df.dropoff_latitude))\
    .filter(
        col("start_cell").isNotNull() & col("end_cell").isNotNull()  # Filter out trips outside of the grid
    )

profitable_df = (
    taxi_df_q2
    .groupBy(window(col("pickup_datetime"), "15 minutes"), col("start_cell"))
    .agg(sum(col("fare_amount") + col("tip_amount")).alias("profit"))
)

taxi_window = Window.partitionBy("medallion").orderBy("pickup_datetime")
freetaxi_df = (
    taxi_df_q2
    .withColumn("next_pickup_datetime", lead("pickup_datetime").over(taxi_window))
    .filter(
        (col("dropoff_datetime").cast("long") >= (col("dropoff_datetime").cast("long") - 1800)) &  # Within the last 30 minutes
        (col("next_pickup_datetime").isNull() | (col("next_pickup_datetime") > col("dropoff_datetime") + expr("INTERVAL 30 MINUTES")))  # No pickup within 30 min
    )
    .groupBy(window(col("dropoff_datetime"), "30 minutes"), col("end_cell"))
    .agg(count("*").alias("available"))
)

profitability_df = (
    freetaxi_df.alias("a")
    .join(profitable_df.alias("p"), col("a.end_cell") == col("p.start_cell"), "inner")
    .select(
        col("p.start_cell"),
        col("a.available"),
        col("p.profit"),
        (col("p.profit") / col("a.available")).alias("profitability")
    )
)

def process(df, epoch_id):
    
    df = df.orderBy(col("profit").desc())

    # Collect unique time windows
    windows = df.select("window").distinct().orderBy("window.start").collect()
    
    # Finding the top routes for each "last 30 minutes"
    for window_row in windows:
        
        time_window = window_row["window"]
        start_time = time_window.start.strftime("%Y-%m-%d %H:%M:%S")
        end_time = time_window.end.strftime("%Y-%m-%d %H:%M:%S")

        display(Markdown(f"Time Window: {start_time} to {end_time}"))
        # Filter the top 10 rides for the given window
        profitable = (
            df.filter(col("window.start") == time_window.start)
            .select("start_cell", "profit", "available", "profit", "profitability")
            .orderBy(col("profitability").desc())
            .limit(10)
            .toPandas()  # Using to Pandas DataFrame to display the results in the notebook
        )

        display(profitable)

query_q2: DataStreamWriter = (
    profitable_df
    .writeStream
    .outputMode("update")
    .foreachBatch(process)
    .start()
)
query_q2.awaitTermination()

AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(dropoff_datetime >= (CAST(dropoff_datetime AS BIGINT) - 1800))" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("TIMESTAMP" and "BIGINT").;
'Filter ((dropoff_datetime#7128 >= (cast(dropoff_datetime#7128 as bigint) - cast(1800 as bigint))) AND (isnull(next_pickup_datetime#7222) OR (next_pickup_datetime#7222 > cast(dropoff_datetime#7128 + INTERVAL '30' MINUTE as timestamp))))
+- Project [medallion#7125, hack_license#7126, pickup_datetime#7127, dropoff_datetime#7128, trip_time_in_secs#7129, trip_distance#7130, pickup_longitude#7131, pickup_latitude#7132, dropoff_longitude#7133, dropoff_latitude#7134, payment_type#7135, fare_amount#7136, surcharge#7137, mta_tax#7138, tip_amount#7139, tolls_amount#7140, start_cell#7158, end_cell#7178, next_pickup_datetime#7222]
   +- Project [medallion#7125, hack_license#7126, pickup_datetime#7127, dropoff_datetime#7128, trip_t

In [ ]:
query_q2.stop()

,start_cell,profit
0,43.330002,8591.02
1,42.340000,4469.20
2,43.340000,3167.58
3,44.320000,2793.43
4,42.349998,2547.63
5,42.330002,2122.11
6,43.320000,2027.42
7,44.330002,1718.69
8,52.380001,1558.87
9,48.330002,1423.22


Time Window: 2013-01-04 14:00:00 to 2013-01-04 14:15:00

,start_cell,profit
0,43.330002,9500.24
1,42.340000,4310.27
2,43.340000,3674.36
3,44.320000,2942.95
4,42.349998,2704.65
5,52.380001,2464.41
6,43.320000,2214.30
7,42.330002,2214.14
8,44.330002,2037.54
9,48.320000,810.42


Time Window: 2013-01-04 14:15:00 to 2013-01-04 14:30:00

,start_cell,profit
0,43.330002,8980.45
1,42.340000,4505.54
2,43.340000,3515.17
3,44.320000,3048.36
4,52.380001,2544.59
5,42.349998,2477.07
6,44.330002,2075.11
7,42.330002,1960.18
8,43.320000,1852.69
9,48.320000,1617.54


...

Time Window: 2013-01-04 18:00:00 to 2013-01-04 18:15:00

,start_cell,profit
0,43.330002,12640.01
1,42.340000,5681.23
2,43.340000,4536.71
3,44.320000,3843.60
4,42.349998,3567.08
5,42.330002,2904.01
6,44.330002,2898.09
7,43.320000,2531.57
8,48.320000,2234.25
9,48.330002,1516.33


Time Window: 2013-01-04 18:15:00 to 2013-01-04 18:30:00

,start_cell,profit
0,43.330002,12869.68
1,42.340000,6860.21
2,43.340000,5616.06
3,42.349998,4150.13
4,44.320000,3260.01
5,42.330002,3075.20
6,44.330002,3062.99
7,52.380001,2693.93
8,43.320000,2282.51
9,48.320000,1354.75


Time Window: 2013-01-04 18:30:00 to 2013-01-04 18:45:00

### Query 2
Profitable Areas

Part I

In [88]:
# Imports 
from pyspark.sql.functions import col, expr, percentile_approx, lit, udf, max, sum, window
from pyspark.sql.types import StringType

# Copy taxi_df dataframe
df = taxi_df.select("*")

def getCellBounds(init_long, init_lat, cellSize):
    change = cellSize/(2 * 111320)
    iters = 300 if cellSize == 500 else 600
    
    leftBorders = [init_long - change]
    topBorders = [init_lat - change]

    for i in range(1, iters):
        leftBorders.append(leftBorders[-1] + 2 * change)
        topBorders.append(topBorders[-1] - 2 * change)

    return (2 * change, leftBorders, topBorders)

def getCellNumber(long, lat, lefts, tops, chg):
    if long < lefts[0] or lat > tops[0] or long > lefts[-1] + chg or lat < tops[-1] - chg:
        return "0.0"
    row = 0
    colNr = 0

    while not lefts[colNr] < long < lefts[colNr] + chg:
        colNr += 1

    while not tops[row] < lat < tops[row] + chg:
        row += 1
    
    return str(row + 1) + '.' + str(colNr + 1)
    
chg, lefts, tops = getCellBounds(-74.913585, 41.474937, 250)
myfunction = udf(getCellNumber, StringType())
# Calculate median profit
# profitability = profit/empty taxis
test = df.withColumns(
    {"startCell": myfunction("pickup_longitude", "pickup_latitude", lit(lefts), lit(tops), lit(chg)),
     "endCell": myfunction("dropoff_longitude", "dropoff_latitude", lit(lefts), lit(tops), lit(chg))}
)

test = test.repartition(100)
test.writeStream \
    .format("memory") \
    .queryName("test") \
    .start()

spark.sql("SELECT * FROM test").show(5, truncate=False)

+---------+------------+---------------+----------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+---------+-------+
|medallion|hack_license|pickup_datetime|dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|startCell|endCell|
+---------+------------+---------------+----------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+---------+-------+
+---------+------------+---------------+----------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+---------+-------+



In [78]:
for q in spark.streams.active:
    q.stop()

In [35]:
latest_dropoff = test.select(max("dropoff_datetime")).collect()[0][0]
print(latest_dropoff)

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[output/preprocessed_data]

In [31]:
# find trips that ended within the last 15 minutes
latest_trips = test.filter((latest_dropoff - col('dropoff_ts')) < 900)
cell_profit = latest_trips.groupBy("startCell").agg(sum(col("fare_amount") + col("tip_amount")))

taxis_with_no_pickups = test.groupBy("medallion").agg(max("dropoff_ts"))
latest_taxis = taxis_with_no_pickups.filter((latest_dropoff - col('max(dropoff_ts)')) < 1800)
latest_taxis = latest_taxis.join(test, (latest_taxis['max(dropoff_ts)'] == test['dropoff_ts']) & (latest_taxis['medallion'] == test['medallion']))
cell_empty_taxis = latest_taxis.groupBy("endCell").count()

joined = cell_profit.join(cell_empty_taxis, cell_profit['startCell'] == cell_empty_taxis['endCell'], how="inner")
joined.show()

NameError: name 'latest_dropoff' is not defined